# Query the live aircraft data from Opensky API

This notebook shows how to query live data from the Opensky API.
https://opensky-network.org/apidoc/

In [1]:
# Columns data from the API
columns = ["icao24",
 "callsign",
 "origin_country",
 "time_position",
 "last_contact",
 "longitude",
 "latitude",
 "baro_altitude",
 "on_ground",
 "velocity",
 "true_track",
 "vertical_rate",
 "sensors",
 "geo_altitude",
 "squawk",
 "spi",
 "position_source",
 "time"]

In [2]:
# Import the libraries that will be used in this notebook
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os

## API configuration and query

In [3]:
# Save config information.
url = "https://opensky-network.org/api/states/all"

# Build partial query URL
query_url = f"{url}"

#  Perform a request for data
response = requests.get(query_url).json()

In [4]:
# Time of the query data
response["time"]

1607213040

In [5]:
# Amount of data point of the query data
len(response["states"])

3554

In [6]:
# One element of the array
response["states"][0]

['a8aac8',
 'DAL1353 ',
 'United States',
 1607213039,
 1607213039,
 -83.1061,
 29.9016,
 11887.2,
 False,
 240.98,
 164.78,
 0,
 None,
 12321.54,
 None,
 False,
 0]

In [7]:
# Map data to the dictorary
aircraft_live_data = [];
for i in range(0, 10): #range(len(response["states"])):
    data = {};
    # Test for null and strip spaces in case not null
    if response["states"][i][0]:
        data["icao24"] = response["states"][i][0].strip()
    else:
        data["icao24"] = response["states"][i][0]
    if response["states"][i][1]:
        data["callsign"] = response["states"][i][1].strip()
    else:
        data["callsign"] = response["states"][i][1]
    if response["states"][i][2]:
        data["origin_country"] = response["states"][i][2].strip()
    else:
        data["origin_country"] = response["states"][i][2]
    data["time_position"] = response["states"][i][3]
    data["last_contact"] = response["states"][i][4]
    data["longitude"] = response["states"][i][5]
    data["latitude"] = response["states"][i][6]
    data["baro_altitude"] = response["states"][i][7]
    data["on_ground"] = response["states"][i][8]
    data["velocity"] = response["states"][i][9]
    data["true_track"] = response["states"][i][10]
    data["vertical_rate"] = response["states"][i][11]
    data["sensors"] = response["states"][i][12]
    data["geo_altitude"] = response["states"][i][13]
    if response["states"][i][14]:
        data["squawk"] = response["states"][i][14].strip()
    else:
        data["squawk"] = response["states"][i][14]
    data["spi"] = response["states"][i][15]
    data["position_source"] = response["states"][i][16]
    data["time"] = response["time"]
    
    aircraft_live_data.append(data)

In [8]:
aircraft_live_data[0]

{'icao24': 'a8aac8',
 'callsign': 'DAL1353',
 'origin_country': 'United States',
 'time_position': 1607213039,
 'last_contact': 1607213039,
 'longitude': -83.1061,
 'latitude': 29.9016,
 'baro_altitude': 11887.2,
 'on_ground': False,
 'velocity': 240.98,
 'true_track': 164.78,
 'vertical_rate': 0,
 'sensors': None,
 'geo_altitude': 12321.54,
 'squawk': None,
 'spi': False,
 'position_source': 0,
 'time': 1607213040}

### Save data to MySQL

In [9]:
import datetime
from datetime import datetime
import mysql.connector
# import database_credentials as dbkeys
import os
import json

with open("/etc/config.json") as config_file:
    config = json.load(config_file)


# Variables:
database_name = "project_2"  # the name of the target database
table_name = "aircraft_data"



# Create a function to connect to the MYSQL server
def database_connect(hostname, username, password, database=database_name):
    mydb = mysql.connector.connect(
        host=hostname,
        user=username,
        passwd=password,
        database=database
    )
    return mydb



# Return an object containing the MYSQL connection
mydb = database_connect(
    config.get("MYSQL_HOSTNAME"),
    config.get("MYSQL_USERNAME"),
    config.get("MYSQL_PASSWORD")
    )
print(mydb)


# Create the cursor to manipute databases
my_cursor = mydb.cursor()
# Add unique constraint


my_cursor.execute(f"SELECT * FROM {database_name}.{table_name} ORDER BY id DESC LIMIT 1;")
for records in my_cursor:
    print(records)
    print(records[0])


# my_cursor.execute(f"""ALTER TABLE `project_2`.`aircraft_data` 
# ADD COLUMN `time` INT NULL DEFAULT NULL COMMENT '' AFTER `position_source`""")

# Create place holders records to insert into the table
sqlStuff = f"""INSERT INTO {table_name} (icao24, 
                                        callsign,
                                        origin_country,
                                        time_position,
                                        last_contact,
                                        longitude,
                                        latitude,
                                        baro_altitude,
                                        on_ground,
                                        velocity,
                                        true_track,
                                        vertical_rate,
                                        sensors,
                                        geo_altitude,
                                        squawk,
                                        spi,
                                        position_source,
                                        time)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """

(749012, 'a41b89', 'DAL2071', 'United States', 1607212799, 1607212799, -83.5589, 32.6071, 11582.4, 0, 199.28, 343.81, 0.0, None, 11902.4, None, 0, 0, None)
749012


In [10]:
for  ii in range(len(aircraft_live_data)):
            record1 = (
                aircraft_live_data[ii]["icao24"],
                aircraft_live_data[ii]["callsign"],
                aircraft_live_data[ii]["origin_country"],
                aircraft_live_data[ii]["time_position"],
                aircraft_live_data[ii]["last_contact"],
                aircraft_live_data[ii]["longitude"],
                aircraft_live_data[ii]["latitude"],
                aircraft_live_data[ii]["baro_altitude"],
                aircraft_live_data[ii]["on_ground"],
                aircraft_live_data[ii]["velocity"],
                aircraft_live_data[ii]["true_track"],
                aircraft_live_data[ii]["vertical_rate"],
                aircraft_live_data[ii]["sensors"],
                aircraft_live_data[ii]["geo_altitude"],
                aircraft_live_data[ii]["squawk"],
                aircraft_live_data[ii]["spi"],
                aircraft_live_data[ii]["position_source"],
                aircraft_live_data[ii]["time"]
                )

#             print(record1)
                

            my_cursor.execute(sqlStuff, record1)

mydb.commit()

In [11]:
for i in range(len(response["states"])):
    aircraft_live_data = list(range(0,18))
    if response["states"][i][0]:
        aircraft_live_data[0] = response["states"][i][0].strip()
    else:
        aircraft_live_data[0] = response["states"][i][0]
    if response["states"][i][1]:
        aircraft_live_data[1] = response["states"][i][1].strip()
    else:
        aircraft_live_data[1] =  response["states"][i][1]
    if response["states"][i][2]:
        aircraft_live_data[2] =  response["states"][i][2].strip()
    else:
        aircraft_live_data[2] =  response["states"][i][2]
    aircraft_live_data[3] =  response["states"][i][3]
    aircraft_live_data[4] =  response["states"][i][4]
    aircraft_live_data[5] =  response["states"][i][5]
    aircraft_live_data[6] =  response["states"][i][6]
    aircraft_live_data[7] =  response["states"][i][7]
    aircraft_live_data[8] =  response["states"][i][8]
    aircraft_live_data[9] =  response["states"][i][9]
    aircraft_live_data[10] =  response["states"][i][10]
    aircraft_live_data[11] =  response["states"][i][11]
    aircraft_live_data[12] =  response["states"][i][12]
    aircraft_live_data[13] =  response["states"][i][13]
    if response["states"][i][14]:
        aircraft_live_data[14] =  response["states"][i][14].strip()
    else:
        aircraft_live_data[14] =  response["states"][i][14]
    aircraft_live_data[15] =  response["states"][i][15]
    aircraft_live_data[16]  = response["states"][i][16]
    aircraft_live_data[17] = response["time"]

In [12]:
# aircraft_live_data

In [13]:
# (aircraft_live_data[0], 
#  aircraft_live_data[1],
#  aircraft_live_data[2],
#  aircraft_live_data[3],
#  aircraft_live_data[4],
#  aircraft_live_data[5],
#  aircraft_live_data[6],
#  aircraft_live_data[7],
#  aircraft_live_data[8],
#  aircraft_live_data[9],
#  aircraft_live_data[10],
#  aircraft_live_data[11],
#  aircraft_live_data[12],
#  aircraft_live_data[13],
#  aircraft_live_data[14],
#  aircraft_live_data[15],
#  aircraft_live_data[16]
# )

### Query Database

In [14]:
my_cursor.execute(f"SELECT * FROM {database_name}.{table_name} ORDER BY id DESC LIMIT 25;")
list_records = []
for records in my_cursor:
#     print(f"Total records on the table: {records[0]}")
    print(f"-----------------------------------------")
#     print(records)
    list_records.append(records)
    

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
----------------------------------

In [15]:
type(records)

tuple

In [16]:
# Convert tuples from database to dataframe
df = pd.DataFrame(list_records, columns = [
                                    "id",
                                    "icao24",
                                     "callsign",
                                     "origin_country",
                                     "time_position",
                                     "last_contact",
                                     "longitude",
                                     "latitude",
                                     "baro_altitude",
                                     "on_ground",
                                     "velocity",
                                     "true_track",
                                     "vertical_rate",
                                     "sensors",
                                     "geo_altitude",
                                     "squawk",
                                     "spi",
                                     "position_source",
                                    "time"]
                 ) 

In [17]:
df.set_index('id')

,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,time
id,,,,,,,,,,,,,,,,,,
749022,ade191,DAL1296,United States,1607213039,1607213039,-80.3799,40.3286,10058.40,0,221.59,13.97,0.33,None,10012.70,None,0,0,1.607213e+09
749021,7c6b41,JST793,Australia,1607213040,1607213040,149.4960,-31.8465,11590.00,0,204.33,219.38,0.00,None,12070.10,1040,0,0,1.607213e+09
749020,7c6b2b,JST737,Australia,1607213039,1607213040,144.6390,-37.7394,3055.62,0,147.65,180.60,12.35,None,2948.94,3257,0,0,1.607213e+09
749019,88044d,AIQ3188,Thailand,1607212985,1607212986,100.4920,13.5793,4053.84,0,193.83,159.98,13.33,None,4274.82,None,0,0,1.607213e+09
749018,ad0b62,LXJ94,United States,1607213040,1607213040,-74.3636,41.9858,4328.16,0,183.69,208.43,0.65,None,4137.66,5105,0,0,1.607213e+09
749017,ac52ee,ASH6273,United States,1607212940,1607212951,-96.1797,30.5337,4953.00,0,160.94,143.97,-8.13,None,5082.54,None,0,0,1.607213e+09
749016,e400d9,FAB2101,Brazil,1607213039,1607213039,-49.1322,-22.0975,8481.06,0,238.61,185.07,6.50,None,9029.70,None,0,0,1.607213e+09
749015,a7f44d,SKW5640,United States,1607213039,1607213039,-97.0840,37.7271,10972.80,0,226.58,279.01,0.00,None,11125.20,None,0,0,1.607213e+09
749014,7c79a4,YBA,Australia,1607212808,1607212808,145.3620,-37.7142,335.28,0,40.34,5.86,-4.55,None,289.56,3402,0,0,1.607213e+09


In [18]:
df.to_dict(orient='records')

[{'id': 749022,
  'icao24': 'ade191',
  'callsign': 'DAL1296',
  'origin_country': 'United States',
  'time_position': 1607213039,
  'last_contact': 1607213039,
  'longitude': -80.3799,
  'latitude': 40.3286,
  'baro_altitude': 10058.4,
  'on_ground': 0,
  'velocity': 221.59,
  'true_track': 13.97,
  'vertical_rate': 0.33,
  'sensors': None,
  'geo_altitude': 10012.7,
  'squawk': None,
  'spi': 0,
  'position_source': 0,
  'time': 1607213040.0},
 {'id': 749021,
  'icao24': '7c6b41',
  'callsign': 'JST793',
  'origin_country': 'Australia',
  'time_position': 1607213040,
  'last_contact': 1607213040,
  'longitude': 149.496,
  'latitude': -31.8465,
  'baro_altitude': 11590.0,
  'on_ground': 0,
  'velocity': 204.33,
  'true_track': 219.38,
  'vertical_rate': 0.0,
  'sensors': None,
  'geo_altitude': 12070.1,
  'squawk': '1040',
  'spi': 0,
  'position_source': 0,
  'time': 1607213040.0},
 {'id': 749020,
  'icao24': '7c6b2b',
  'callsign': 'JST737',
  'origin_country': 'Australia',
  'time_